In [5]:
# Question 2 / 3

import cs111
import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

# Question 2
def S(x,y):
    return 2*np.cos(x) + 2*np.sin(y)

def boundary_condition(x, y):
    return np.cos(x) + np.sin(y) # we use the exact solution given in 1.3

def make_linear_system(xmin, xmax, ymin, ymax, mX, mY, k): # we add k as a variable 
    """Create the matrix of the discrete Laplacian operator in two dimensions on a mX-by-mY grid.
    Parameters:
      :param xmin: interval [xmin, xmax]
      :param xmax: interval [xmin, xmax]
      :param ymin: interval [ymin, ymax]
      :param ymax: interval [ymin, ymax]
      :param mX  : number of grid points in the x-direction.
      :param mY  : number of grid points in the y-direction.
    Outputs:
      A  : the sparse mX*mY-by-mX*mY matrix representing the finite difference approximation to Laplace's equation.
      rhs: the right-hand side of the linear system

    """
    # Define the grid
    x = np.linspace(xmin, xmax, mX)
    y = np.linspace(ymin, ymax, mY)

    dx = x[1] - x[0]
    dy = y[1] - y[0]

    one_over_dx_square = 1 / dx / dx
    one_over_dy_square = 1 / dy / dy

    # Start with a vector of zeros
    ndim = mX * mY
    rhs = np.zeros(shape=ndim)

    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []

    # Treat the interior points:
    for i in range(1, mX - 1):
        for j in range(1, mY - 1):
            # what row of the matrix is grid point (i, j)?
            row = j*mX + i 
            # fill out the row in the matrix:
            triples.append((row, row, 2.0 * one_over_dx_square + 2.0 * one_over_dy_square + k))# Since there is a kui,j, it needs to be added in to the coefficent
            triples.append((row, row - 1, -1.0 * one_over_dx_square))
            triples.append((row, row + 1, -1.0 * one_over_dx_square))
            triples.append((row, row - mX, -1.0 * one_over_dy_square))
            triples.append((row, row + mX, -1.0 * one_over_dy_square))
            rhs[row] = S(x[i], y[j])

    # Treat the boundary points:
    for i in range(mX):
        j = 0  # bottom wall
        row = j*mX + i
        triples.append((row, row, 1.0))
        rhs[row] = boundary_condition(x[i], y[j])
        j = mY - 1  # top wall
        row = j*mX + i
        triples.append((row, row, 1.0))
        rhs[row] = boundary_condition(x[i], y[j])

    for j in range(1, mY - 1):  # Need to avoid 0 and mY-1 because duplicate entries are summed up with csr_matrix
        i = 0  # left wall
        row = j*mX + i
        triples.append((row, row, 1.0))
        rhs[row] = boundary_condition(x[i], y[j])
        i = mX - 1  # right wall
        row = j*mX + i
        triples.append((row, row, 1.0))
        rhs[row] = boundary_condition(x[i], y[j])

    # Finally convert the list of triples to a scipy sparse matrix
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = scipy.sparse.csr_matrix((values, (rownum, colnum)), shape=(ndim, ndim))

    return A, rhs


#Question 3 
mX = 50
mY = 50
xmin = 0; xmax = 1; ymin = 0; ymax = 1;k = 1 
# Build the linear system of equations:
A, b = make_linear_system(xmin, xmax, ymin, ymax, mX, mX,k)
#B, c = make_linear_system(0,1,0,1,100,100,1)

#C = B - A
#print(np.all(C==0))

u = scipy.sparse.linalg.spsolve(A, b)

def u_exact(x, y):
    return np.cos(x) + np.sin(y)

x = np.linspace(xmin, xmax, mX)
y = np.linspace(ymin, ymax, mY)
X, Y = np.meshgrid(x, y)
u_exact = u_exact(X, Y)
u = u.reshape( mX, mY )
max_error = np.amax(np.abs(u_exact - u))
print("maximum error is ", max_error)

print(u)

maximum error is  3.2604669786273632e-06
[[1.         0.99979176 0.99916713 ... 0.57418852 0.55736148 0.54030231]
 [1.02040675 1.02019854 1.01957394 ... 0.59459532 0.57776825 0.56070905]
 [1.04080499 1.04059682 1.03997225 ... 0.61499361 0.59816652 0.5811073 ]
 ...
 [1.81872312 1.81851497 1.81789042 ... 1.39291179 1.37608467 1.35902542]
 [1.83026995 1.83006176 1.82943717 ... 1.40445855 1.38763147 1.37057225]
 [1.84147098 1.84126275 1.84063811 ... 1.41565951 1.39883246 1.38177329]]


In [13]:
# Question 4 
#change the S into 0 and modify the u after solving. Plot
